# Louiville space
TODO: write a class to create a superoperator from a given 
operator, and to transform a density matrix into a column vector, properly stripping the redundant elements of each.

In [2]:
from numpy import *
from sympy import symbols

In [4]:
rho = array([[1,2],[3,4]])
rho.flatten()

array([1, 2, 3, 4])